<a href="https://colab.research.google.com/github/sabithajaleel/ML-Notebooks/blob/main/DubaiRecommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install lat-lon-parser

In [20]:
from bs4 import BeautifulSoup # library to extract data from a website and used for analytics
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
import requests
from lat_lon_parser import parse

touristAttractionsData = pd.DataFrame(columns=["City", "Attractions", "Latitude", "Longitude"])

r = requests.get("https://en.wikipedia.org/wiki/Tourist_attractions_in_Dubai")

soup = BeautifulSoup(r.text.replace('\n', ''), "html.parser")

lis = soup.find("div", {"class": "mw-content-ltr"}).find("ul").find_all("li")

for li in lis:
  link = li.find("a", href=True)

  attraction = link["title"]

  r = requests.get("https://en.wikipedia.org" + link["href"])
  soup = BeautifulSoup(r.text.replace('\n', ''), "html.parser")

  latitudestr = soup.find('span', {"class": "latitude"}).text
  longitudestr = soup.find('span', {"class": "longitude"}).text
  latitude = parse(latitudestr)
  longitude = parse(longitudestr)

  touristAttractionsData = touristAttractionsData.append({
      "City": "Dubai",
      "Attractions": attraction,
      "Latitude": float(latitude),
      "Longitude": float(longitude)
  }, ignore_index=True)

touristAttractionsData.head()

<ipython-input-20-605116487bb2>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  touristAttractionsData = touristAttractionsData.append({
<ipython-input-20-605116487bb2>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  touristAttractionsData = touristAttractionsData.append({
<ipython-input-20-605116487bb2>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  touristAttractionsData = touristAttractionsData.append({
<ipython-input-20-605116487bb2>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  touristAttractionsData = touristAttractionsData.append({
<ipython-input-20-605116487bb2>:28: FutureWarning: The frame.append method is deprecated

,City,Attractions,Latitude,Longitude
0,Dubai,Burj Khalifa,25.197222,55.274167
1,Dubai,The Dubai Fountain,25.195278,55.275000
2,Dubai,Dubai Frame,25.235556,55.300278
3,Dubai,Museum of the Future,25.219119,55.282100
4,Dubai,Dubai Miracle Garden,25.059722,55.244444


In [59]:
# call Foursquare API for each location and get the nearby venues, within a 1000-meter radius of that location.
import requests

def getNearbyVenues(names, latitudes, longitudes, radius=1000):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url = "https://api.foursquare.com/v3/places/search"

        latlongarr = [str(lat), str(longitude)]
        params = {
            # "query": "Coffee",
  	        "ll": ",".join(latlongarr),
  	        "open_now": "true",
  	        "sort":"DISTANCE"
            }

        headers = {
            "Accept": "application/json",
            "Authorization": "fsq3jJaXevy7ImJxh08qwY/cj25mZLHoVjz5rQWssp/zFPI="
        }

        response = requests.request("GET", url, params=params, headers=headers)
        results = response.json()["results"]
        if results:
          for v in results:
           venues_list.append([(
            name, lat, lng, v['categories'][0]['name'], v['categories'][0]['id'],
           v['geocodes']['main']['latitude'], v['geocodes']['main']['longitude'],
           v['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood      Longitude',  'Venue', 'id', 'Venue Latitude', 'Venue Longitude','Venue Category']

    return(nearby_venues)


In [60]:
yasIsland = touristAttractionsData[touristAttractionsData['Attractions'] == 'Yas Island'].reset_index(drop=True)
attraction_venues = getNearbyVenues(names=yasIsland['Attractions'],
                                   latitudes=yasIsland['Latitude'],
                                   longitudes=yasIsland['Longitude']
                                  )
attraction_venues.head()

Yas Island


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,id,Venue Latitude,Venue Longitude,Venue Category
0,Yas Island,24.5,54.6,Amusement Park,10001,24.494588,54.595676,Warner Bros. World Abu Dhabi: Address
1,Yas Island,24.5,54.6,Amusement Park,10001,24.488814,54.600450,Yas Waterworld
2,Yas Island,24.5,54.6,Arcade,10003,24.490397,54.607834,Fun Works - Yas Mall
3,Yas Island,24.5,54.6,Department Store,17033,24.489162,54.606710,Tryano
4,Yas Island,24.5,54.6,Drugstore,17035,24.488402,54.606442,Boots


In [61]:
attraction_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Neighbourhood                 100 non-null    object 
 1   Neighbourhood Latitude        100 non-null    float64
 2   Neighbourhood      Longitude  100 non-null    float64
 3   Venue                         100 non-null    object 
 4   id                            100 non-null    int64  
 5   Venue Latitude                100 non-null    float64
 6   Venue Longitude               100 non-null    float64
 7   Venue Category                100 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 6.4+ KB
